# Generate species tree

In [ ]:
from pathlib import Path
import subprocess
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test")

In [ ]:
sp_tree_index = 0
complete_tree = OUTPUT_FOLDER / f"species_tree_{sp_tree_index}" / "complete_species_tree.nwk"
extant_tree = OUTPUT_FOLDER / f"species_tree_{sp_tree_index}" / "extant_species_tree.nwk"

In [ ]:
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/Zombi_ale_friendly/Parameters/SpeciesTreeParameters.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

# Generate gene trees

In [ ]:
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/Zombi_ale_friendly/Parameters/GenomeParameters.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

# Sampling avec Zombi

## Sampling uniforme

In [ ]:
# sampled_species = 5
# command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/SpeciesSampler.py n {sampled_species} {tree_output_folder}"
# subprocess.run(command, shell=True)

## Sampling depuis une liste d'espèces

In [ ]:
from ete3 import Tree
import random
sampled_species_path = "/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/sampled_leaves_list.txt"
extant_tree = Tree("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_0/T/ExtantTree.nwk", format = 1)
list_leaf_names = [leaf.name for leaf in extant_tree.get_leaves()]
sampled_species_names = random.sample(list_leaf_names, 4)
print(f"sampled_species {sampled_species_names}")
with open(sampled_species_path, "w") as f:
    # write the list of species
    for species in sampled_species_names:
        f.write(f"{species}\n")

In [ ]:
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/SpeciesSampler.py i {sampled_species_path} {tree_output_folder}"
subprocess.run(command, shell=True)

# Autres tests de sampling

In [ ]:
import ete3
import random
from ete3 import Tree

# on commence par faire les versions les plus simples, et on les teste au lieu d'ajouter tout de suite une abstraction

def sampling_diversified(tree, n_sampled, seed=1729):
    # We follow the sampling strategies in the paper
    # Inferring Speciation and Extinction Rates under Different Sampling Schemes by Höhna et al.
    # The paper defines the diversified sampling strategy as maximizing the total branch length of the sampled species.
    # This in turn is equivalent to taking the n_sampled-1 oldest bifurcations in the tree.
    # To do that, we sort the bifurcations. Then we successively add nodes such that the new bifurcation is the youngest one
    # among the bifurcations we must pick.


    random.seed(seed)
    sampled_leaves = []
    node_depths = dict()
    # Add depth attribute to each node
    for node in tree.traverse():
        if node.is_root():
            node.depth = 0
        else:
            node.depth = node.up.depth + node.dist
        node_depths[node] = node.depth

    # Sort nodes by depth
    nodes = sorted(node_depths.keys(), key=lambda x: x.depth, reverse=True)
    # Now we want the n_sampled-1 oldest bifurcations
    sampled_internal_nodes = nodes[:n_sampled]
    # We need to make sure we have a leaf for each side of each chosen internal node. We can ensure that by starting with the least deep internal node.



In [ ]:
!pip install ete3

In [ ]:
from ete3 import Tree
t = Tree()
t.populate(10)

In [ ]:
t.get_children()[0].get_leaves()

In [ ]:
import random
from ete3 import Tree

def sampling(tree, n_sampled, seed=1729, strategy="uniform"):
    """
    Implements a diversified sampling strategy using a two-pass approach.
    
    First, it selects the deepest (oldest) n_sampled-1 non-overlapping internal nodes.
    Then, it builds a dictionary mapping each chosen internal node to a tuple of booleans,
    indicating for each child whether that child is itself chosen.
    In the second pass, for every child marked False (i.e. not chosen as an internal node),
    a representative leaf (picked at random from that child's descendant leaves) is added.
    
    The final list of sampled leaves should have size n_sampled.
    
    Parameters:
      tree      : an ete3 Tree instance.
      n_sampled : the desired number of sampled leaves.
      seed      : random seed for reproducibility.
      
    Returns:
      A list of sampled leaf nodes.
    """
    random.seed(seed)

    # Pass 0: Assign cumulative depth (distance from root) to every node.
    for node in tree.traverse("preorder"):
        if node.is_root():
            node.add_features(depth=0)
        else:
            node.add_features(depth=node.up.depth + node.dist)
    if strategy == "uniform":
        # Choose leaves randomly.
        leaves = tree.get_leaves()
        sampled_leaves = random.sample(leaves, n_sampled)
    else:
        # Pass 1: Select nonoverlapping internal nodes (excluding the root) by descending depth.
        internals = [node for node in tree.traverse() if (not node.is_leaf())]

        internals.sort(key=lambda n: n.depth, reverse=False)
        if strategy == "diversified":
            chosen_internals = internals[:n_sampled - 1]
        elif strategy == "clustered":
            chosen_internals = internals[-n_sampled + 1:]
        else:
            raise ValueError("Invalid sampling strategy.")
        chosen_set = set(chosen_internals)
        
        # Build a dictionary mapping chosen internal nodes to a tuple of booleans.
        # Each boolean corresponds to a child: True if the child is also chosen, False otherwise.
        chosen_dict = {}
        for node in chosen_internals:
            children = node.get_children()
            flags = tuple(child in chosen_set for child in children)
            chosen_dict[node] = flags

        # Pass 2: For each chosen internal node, for each child not chosen, pick a random leaf from that branch.
        sampled_leaves = []
        for node, flags in chosen_dict.items():
            children = node.get_children()
            for flag, child in zip(flags, children):
                if not flag:
                    # Pick a random leaf from this child's descendants.
                    leaves = child.get_leaves()
                    if leaves:
                        sampled_leaves.append(random.choice(leaves))
    
    return sampled_leaves

# Example usage:
if __name__ == '__main__':
    # Example Newick tree (binary tree)
    newick = "((A:0.1,B:0.2):0.3,((C:0.2,D:0.3):0.4,E:0.5):0.6);"
    tree = Tree(newick, format=1)
    n_sampled = 4
    sampled = sampling(tree, n_sampled=n_sampled, seed=11110, strategy="clustered")
    print("Sampled leaves:")
    for leaf in sampled:
        print(leaf.name, "Depth:", leaf.depth)


In [ ]:
# Clustered does not work yet.

In [ ]:
# verify that leaf C appears in 50% of samples:
c_appearances = 0
for seed in range(100000):
    sampled = sampling(tree, n_sampled=4, seed=seed, strategy="diversified")
    if "C" in [leaf.name for leaf in sampled]:
        c_appearances += 1
print(c_appearances / 100000)

In [ ]:
tree.write(outfile="tree.nwk", format=1)

In [ ]:
from ete3 import Tree
import itertools
import copy

def min_k_leaf_subtree(tree, k):
    """
    Find a subset of k leaves such that the pruned tree has minimal branch length sum.
    
    Args:
        tree: An ete3.Tree object representing the phylogenetic tree
        k: Number of leaves to keep in the pruned tree
    
    Returns:
        best_leaves: List of the k leaf names to keep
        min_length: The minimum total branch length of the pruned tree
        pruned_tree: The pruned tree with the k selected leaves
    """
    # Get all leaf nodes
    all_leaves = tree.get_leaves()
    leaf_names = [leaf.name for leaf in all_leaves]
    
    # Check if k is valid
    if k <= 0 or k > len(all_leaves):
        raise ValueError(f"k must be between 1 and {len(all_leaves)}")
    
    # If k equals the number of leaves, return the original tree
    if k == len(all_leaves):
        return leaf_names, tree.get_distance(tree), copy.deepcopy(tree)
    
    min_length = float('inf')
    best_leaves = None
    best_pruned_tree = None
    
    # Try all possible combinations of k leaves
    for leaf_subset in itertools.combinations(leaf_names, k):
        # Create a copy of the tree to prune
        pruned_tree = copy.deepcopy(tree)
        
        # Create a set of leaves to keep
        leaves_to_keep = set(leaf_subset)
        
        # Get all leaves in the copied tree
        pruned_leaves = pruned_tree.get_leaves()
        
        # Prune leaves not in our subset
        for leaf in pruned_leaves:
            if leaf.name not in leaves_to_keep:
                leaf.delete()
        
        # Calculate total branch length
        total_length = sum(node.dist for node in pruned_tree.traverse())
        
        # Update if this is better than our current best
        if total_length < min_length:
            min_length = total_length
            best_leaves = leaf_subset
            best_pruned_tree = pruned_tree
    
    return best_leaves, min_length, best_pruned_tree


def min_k_leaf_subtree_greedy(tree, k):
    """
    Find a subset of k leaves using a greedy algorithm.
    This is more efficient for large trees but doesn't guarantee optimality.
    
    Args:
        tree: An ete3.Tree object representing the phylogenetic tree
        k: Number of leaves to keep in the pruned tree
    
    Returns:
        best_leaves: List of the k leaf names to keep
        total_length: The total branch length of the pruned tree
        pruned_tree: The pruned tree with the k selected leaves
    """
    # Get all leaf nodes
    all_leaves = tree.get_leaves()
    leaf_names = [leaf.name for leaf in all_leaves]
    
    # Check if k is valid
    if k <= 0 or k > len(all_leaves):
        raise ValueError(f"k must be between 1 and {len(all_leaves)}")
    
    # If k equals the number of leaves, return the original tree
    if k == len(all_leaves):
        return leaf_names, tree.get_distance(tree), copy.deepcopy(tree)
    
    # Start with all leaves and remove them one by one
    leaves_to_remove = len(all_leaves) - k
    current_tree = copy.deepcopy(tree)
    current_leaves = current_tree.get_leaves()
    
    for _ in range(leaves_to_remove):
        best_removal_length = float('inf')
        best_leaf_to_remove = None
        
        # Try removing each leaf and see which one results in the smallest tree
        for leaf in current_leaves:
            # Make a copy of the current tree
            test_tree = copy.deepcopy(current_tree)
            
            # Find the corresponding leaf in the test tree
            test_leaf = test_tree.search_nodes(name=leaf.name)[0]
            
            # Remove this leaf
            test_leaf.delete()
            
            # Calculate the total branch length
            test_length = sum(node.dist for node in test_tree.traverse())
            
            # Update if this is better
            if test_length < best_removal_length:
                best_removal_length = test_length
                best_leaf_to_remove = leaf.name
        
        # Remove the best leaf from our current tree
        leaf_to_remove = current_tree.search_nodes(name=best_leaf_to_remove)[0]
        leaf_to_remove.delete()
        
        # Update the current leaves
        current_leaves = current_tree.get_leaves()
    
    # Get the names of the remaining leaves
    remaining_leaves = [leaf.name for leaf in current_leaves]
    total_length = sum(node.dist for node in current_tree.traverse())
    
    return remaining_leaves, total_length, current_tree


def min_k_leaf_subtree_dp(tree, k):
    """
    A dynamic programming approach for the minimum k-leaf subtree problem.
    This works well for trees with special structure (like ultrametric trees).
    
    This implementation assumes binary trees for simplicity.
    
    Args:
        tree: An ete3.Tree object representing the phylogenetic tree
        k: Number of leaves to keep in the pruned tree
    
    Returns:
        best_leaves: List of the k leaf names to keep
        min_length: The minimum total branch length
        pruned_tree: The pruned tree with k selected leaves
    """
    # Check if tree is binary
    for node in tree.traverse():
        if not node.is_leaf() and len(node.children) != 2:
            raise ValueError("This DP approach requires a binary tree")
    
    # Dictionary to store optimal results for subtrees
    memo = {}
    
    def dp(node, leaves_to_keep):
        """DP helper function that computes the minimum branch length
        for keeping a certain number of leaves in the subtree rooted at node."""
        
        # Base cases
        if node.is_leaf():
            if leaves_to_keep == 1:
                return 0, [node.name]  # Keep this leaf, no additional length
            elif leaves_to_keep == 0:
                return 0, []  # Don't keep this leaf
            else:
                return float('inf'), []  # Invalid: can't keep more leaves than exist
        
        # Check if we've solved this subproblem before
        if (id(node), leaves_to_keep) in memo:
            return memo[(id(node), leaves_to_keep)]
        
        # If we need to keep 0 leaves, prune this entire subtree
        if leaves_to_keep == 0:
            return 0, []
        
        # Get the two children
        left_child, right_child = node.children
        
        # Initialize best result
        min_length = float('inf')
        best_leaves = []
        
        # Try all ways to distribute the leaves_to_keep between the two children
        for left_leaves in range(leaves_to_keep + 1):
            right_leaves = leaves_to_keep - left_leaves
            
            # Skip invalid distributions
            if right_leaves < 0:
                continue
            
            # Recursive calls for left and right subtrees
            left_length, left_selected = dp(left_child, left_leaves)
            right_length, right_selected = dp(right_child, right_leaves)
            
            # Calculate total length including the current node's branches
            current_length = left_length + right_length
            
            # Add branch lengths to children only if we're keeping at least one leaf in that subtree
            if left_leaves > 0:
                current_length += left_child.dist
            if right_leaves > 0:
                current_length += right_child.dist
            
            # Update if better than current best
            if current_length < min_length:
                min_length = current_length
                best_leaves = left_selected + right_selected
        
        # Memoize and return the result
        memo[(id(node), leaves_to_keep)] = (min_length, best_leaves)
        return min_length, best_leaves
    
    # Start DP from the root
    total_min_length, selected_leaves = dp(tree, k)
    
    # Create the pruned tree
    pruned_tree = copy.deepcopy(tree)
    for leaf in pruned_tree.get_leaves():
        if leaf.name not in selected_leaves:
            leaf.delete()
    
    return selected_leaves, total_min_length, pruned_tree


# Example usage
def example():
    # Create a simple phylogenetic tree
    newick_str = "((A:9,B:9):2,(C:4,(D:2,E:2):2):7):2;"
    tree = Tree(newick_str)
    
    # Print the original tree
    print("Original tree:")
    print(tree.get_ascii(show_internal=True))
    
    # Find the minimum 3-leaf subtree
    k = 3
    print(f"\nFinding minimum {k}-leaf subtree:")
    
    # Use the exhaustive approach for small trees
    best_leaves, min_length, pruned_tree = min_k_leaf_subtree(tree, k)
    print(f"Optimal leaves to keep: {best_leaves}")
    print(f"Minimum branch length sum: {min_length}")
    print("Pruned tree:")
    print(pruned_tree.get_ascii(show_internal=True))
    
    # Compare with the greedy approach
    greedy_leaves, greedy_length, greedy_tree = min_k_leaf_subtree_greedy(tree, k)
    print(f"\nGreedy approach leaves: {greedy_leaves}")
    print(f"Greedy approach length: {greedy_length}")
    print("Greedy pruned tree:")
    print(greedy_tree.get_ascii(show_internal=True))
    
    # Try the DP approach for binary trees
    dp_leaves, dp_length, dp_tree = min_k_leaf_subtree_dp(tree, k)
    print(f"\nDP approach leaves: {dp_leaves}")
    print(f"DP approach length: {dp_length}")
    print("DP pruned tree:")
    print(dp_tree.get_ascii(show_internal=True))
    tree.write(outfile="tree.nwk", format=1)

if __name__ == "__main__":
    example()

In [ ]:
tree.write(outfile="tree.nwk", format=1)

In [ ]:
import numpy as np
from ete3 import Tree

def min_pruned_tree(tree, k):
    """
    Find a subset of k leaves in an ultrametric phylogenetic tree 
    that minimizes the sum of branch lengths in the pruned tree.
    
    Args:
        tree (ete3.Tree): An ultrametric phylogenetic tree
        k (int): Number of leaves to keep
        
    Returns:
        tuple: (minimum sum of branch lengths, list of selected leaf names)
    """
    # First ensure we're working with an ultrametric tree
    if not is_ultrametric(tree):
        raise ValueError("Input tree must be ultrametric")
    
    # Get all leaf nodes
    leaves = tree.get_leaves()
    total_leaves = len(leaves)
    
    if k > total_leaves:
        raise ValueError(f"k ({k}) cannot be greater than the number of leaves ({total_leaves})")
    
    if k == total_leaves:
        # If k equals the number of leaves, return the original tree's branch length sum
        return tree_length(tree), [leaf.name for leaf in leaves]
    
    # Create a post-order traversal of the tree
    post_order_nodes = list(tree.traverse("postorder"))
    
    # Initialize memoization table
    # dp[node_idx][j] = min branch length sum when selecting j leaves from subtree rooted at node_idx
    dp = {}
    selected_leaves = {}  # To track which leaves are selected
    
    # Post-order traversal to fill the dp table
    for node in post_order_nodes:
        node_idx = id(node)
        dp[node_idx] = {}
        selected_leaves[node_idx] = {}
        
        if node.is_leaf():
            # Base case: leaf node
            dp[node_idx][0] = 0  # No leaves selected = 0 branch length
            dp[node_idx][1] = 0  # One leaf selected = 0 additional branch length for ultrametric tree
            selected_leaves[node_idx][0] = []
            selected_leaves[node_idx][1] = [node.name]
        else:
            # Internal node
            children = node.children
            
            # Initialize with "infinity" for this subtree
            for j in range(k + 1):
                dp[node_idx][j] = float('inf')
                selected_leaves[node_idx][j] = []
            
            # Base case: no leaves selected from this subtree
            dp[node_idx][0] = 0
            selected_leaves[node_idx][0] = []
            
            if len(children) == 2:  # Binary tree
                left_child, right_child = children
                left_idx, right_idx = id(left_child), id(right_child)
                
                # Calculate max possible leaves from each child
                max_left_leaves = len(left_child.get_leaves())
                max_right_leaves = len(right_child.get_leaves())
                
                # For each possible number of leaves to select from this subtree
                for j in range(1, min(k + 1, total_leaves + 1)):
                    # Try different combinations of leaves from left and right children
                    for left_leaves in range(min(j + 1, max_left_leaves + 1)):
                        right_leaves = j - left_leaves
                        
                        if right_leaves > max_right_leaves or right_leaves < 0:
                            continue
                        
                        # Skip if we don't have solutions for these subproblems
                        if left_leaves not in dp[left_idx] or right_leaves not in dp[right_idx]:
                            continue
                        
                        # Calculate branch length for this combination
                        branch_sum = dp[left_idx][left_leaves] + dp[right_idx][right_leaves]
                        
                        # Add the branch length from the current node to its children
                        # But only if we're selecting at least one leaf from each child
                        if left_leaves > 0:
                            branch_sum += left_child.dist
                        if right_leaves > 0:
                            branch_sum += right_child.dist
                            
                        # If this is better than our current best
                        if branch_sum < dp[node_idx][j]:
                            dp[node_idx][j] = branch_sum
                            selected_leaves[node_idx][j] = (
                                selected_leaves[left_idx][left_leaves] + 
                                selected_leaves[right_idx][right_leaves]
                            )
            else:
                # Handle non-binary trees by considering all possible combinations
                # This is a generalization of the binary tree case
                combinations = generate_leaf_combinations(children, j, dp, selected_leaves)
                for combo in combinations:
                    branch_sum = sum(dp[id(child)][combo[i]] for i, child in enumerate(children))
                    
                    # Add branch length for each child with selected leaves
                    for i, child in enumerate(children):
                        if combo[i] > 0:
                            branch_sum += child.dist
                    
                    if branch_sum < dp[node_idx][j]:
                        dp[node_idx][j] = branch_sum
                        selected_leaves[node_idx][j] = []
                        for i, child in enumerate(children):
                            selected_leaves[node_idx][j].extend(selected_leaves[id(child)][combo[i]])
    
    # Return the minimum branch length sum and the selected leaves
    return dp[id(tree)][k], selected_leaves[id(tree)][k]


def generate_leaf_combinations(children, j, dp, selected_leaves):
    """
    Generate all valid combinations of selecting j leaves from the given children.
    
    Args:
        children: List of child nodes
        j: Number of leaves to select
        dp: Dynamic programming table
        selected_leaves: Dictionary to track selected leaves
        
    Returns:
        List of valid combinations
    """
    from itertools import product
    
    # Get the maximum number of leaves for each child
    max_leaves = [len(child.get_leaves()) for child in children]
    
    # Generate all possible combinations
    valid_combinations = []
    
    # Function to generate valid partitions recursively
    def generate_partitions(child_idx, remaining, current_combo):
        if child_idx == len(children):
            if remaining == 0:
                valid_combinations.append(current_combo[:])
            return
        
        # Try different number of leaves for current child
        max_for_child = min(max_leaves[child_idx], remaining)
        
        for i in range(max_for_child + 1):
            # Check if this subproblem has a solution
            if i in dp[id(children[child_idx])]:
                current_combo.append(i)
                generate_partitions(child_idx + 1, remaining - i, current_combo)
                current_combo.pop()
    
    # Start the recursive generation
    generate_partitions(0, j, [])
    
    return valid_combinations


def is_ultrametric(tree, tolerance=1e-10):
    """
    Check if a tree is ultrametric (all leaves are equidistant from the root).
    
    Args:
        tree (ete3.Tree): A phylogenetic tree
        tolerance (float): Tolerance for floating point comparisons
        
    Returns:
        bool: True if the tree is ultrametric, False otherwise
    """
    leaves = tree.get_leaves()
    if not leaves:
        return True
    
    # Get the distance from root to the first leaf
    reference_dist = leaves[0].get_distance(tree)
    
    # Check if all leaves are at the same distance from the root
    for leaf in leaves[1:]:
        dist = leaf.get_distance(tree)
        if abs(dist - reference_dist) > tolerance:
            return False
    
    return True


def tree_length(tree):
    """
    Calculate the sum of all branch lengths in a tree.
    
    Args:
        tree (ete3.Tree): A phylogenetic tree
        
    Returns:
        float: Sum of all branch lengths
    """
    return sum(node.dist for node in tree.traverse() if node is not tree)


def prune_tree_with_leaves(tree, leaf_names):
    """
    Prune a tree to keep only the specified leaves.
    
    Args:
        tree (ete3.Tree): A phylogenetic tree
        leaf_names (list): List of leaf names to keep
        
    Returns:
        ete3.Tree: Pruned tree
    """
    # Create a copy of the tree to avoid modifying the original
    pruned_tree = tree.copy()
    
    # Get all leaves
    leaves = pruned_tree.get_leaves()
    
    # Determine which leaves to remove
    leaves_to_remove = [leaf for leaf in leaves if leaf.name not in leaf_names]
    
    # Remove unwanted leaves
    for leaf in leaves_to_remove:
        leaf.delete()
    
    return pruned_tree


# Example usage
def generate_yule_tree(n_leaves, birth_rate=1.0, seed=None):
    """
    Generate a pure-birth (Yule) process tree with n_leaves.
    
    Args:
        n_leaves (int): Number of leaves in the final tree
        birth_rate (float): Rate of speciation events
        seed (int): Random seed for reproducibility
        
    Returns:
        ete3.Tree: An ultrametric phylogenetic tree
    """
    import random
    
    if seed is not None:
        random.seed(seed)
    
    # Start with a single node (root)
    tree = Tree()
    tree.name = "Root"
    
    # Add the first two leaves (first speciation event)
    time_elapsed = random.expovariate(birth_rate)
    leaf1 = tree.add_child(name="Leaf1")
    leaf1.dist = time_elapsed
    leaf2 = tree.add_child(name="Leaf2")
    leaf2.dist = time_elapsed
    
    # Current leaves that can speciate
    active_leaves = [leaf1, leaf2]
    leaf_counter = 3  # Counter for naming new leaves
    
    # Continue until we reach n_leaves
    while len(active_leaves) < n_leaves:
        # Choose a random leaf to speciate
        parent = random.choice(active_leaves)
        active_leaves.remove(parent)
        
        # Generate waiting time until next speciation
        waiting_time = random.expovariate(birth_rate * len(active_leaves))
        
        # Update all active leaf distances (simulation moves forward in time)
        for leaf in active_leaves:
            leaf.dist += waiting_time
        
        # Create two new leaves
        child1 = parent.add_child(name=f"Leaf{leaf_counter}")
        leaf_counter += 1
        child2 = parent.add_child(name=f"Leaf{leaf_counter}")
        leaf_counter += 1
        
        # Set the distances for new leaves to 0 (they'll accumulate time as we proceed)
        child1.dist = 0
        child2.dist = 0
        
        # Add new leaves to active set
        active_leaves.append(child1)
        active_leaves.append(child2)
    
    # Make sure the tree is ultrametric by adjusting terminal branch lengths
    leaves = tree.get_leaves()
    max_depth = max(leaf.get_distance(tree) for leaf in leaves)
    
    for leaf in leaves:
        current_depth = leaf.get_distance(tree)
        if current_depth < max_depth:
            leaf.dist += (max_depth - current_depth)
    
    # Verify ultrametricity
    assert is_ultrametric(tree), "Generated tree is not ultrametric"
    
    return tree


if __name__ == "__main__":
    # First test with a simple predefined tree
    print("TEST 1: Simple predefined tree")
    newick_str = "((A:1,B:1):2,(C:1.5,D:1.5):1.5);"
    tree = Tree(newick_str)
    
    # Find the minimum pruned tree with 3 leaves
    k = 3
    min_length, selected_leaves = min_pruned_tree(tree, k)
    
    print(f"Minimum branch length: {min_length}")
    print(f"Selected leaves: {selected_leaves}")
    
    # Prune the tree to keep only the selected leaves
    pruned_tree = prune_tree_with_leaves(tree, selected_leaves)
    
    # Print the pruned tree
    print("Pruned tree:")
    print(pruned_tree.get_ascii())
    
    # Verify the branch length
    actual_length = tree_length(pruned_tree)
    print(f"Actual branch length of pruned tree: {actual_length}")
    
    # Now generate and test with a Yule tree
    print("\nTEST 2: Generated Yule tree")
    n_leaves = 20
    k = 4
    
    # Generate a random Yule tree
    yule_tree = generate_yule_tree(n_leaves, seed=420)
    print(f"Generated Yule tree with {len(yule_tree.get_leaves())} leaves")
    
    # Verify ultrametricity
    print(f"Tree is ultrametric: {is_ultrametric(yule_tree)}")
    
    # Calculate total branch length of original tree
    original_length = tree_length(yule_tree)
    print(f"Original tree branch length: {original_length}")
    
    # Find minimum pruned tree
    print(f"Finding minimum pruned tree with {k} leaves...")
    min_length, selected_leaves = min_pruned_tree(yule_tree, k)
    print(f"Minimum branch length: {min_length}")
    print(f"Selected leaves: {selected_leaves}")
    
    # Prune the tree
    pruned_tree = prune_tree_with_leaves(yule_tree, selected_leaves)
    yule_tree.write(outfile="yule_tree.nwk", format=1)
    # Verify the pruned tree has exactly k leaves
    pruned_leaves = pruned_tree.get_leaves()
    print(f"Pruned tree has {len(pruned_leaves)} leaves")
    
    # Verify ultrametricity of pruned tree
    print(f"Pruned tree is ultrametric: {is_ultrametric(pruned_tree)}")
    
    # Calculate actual branch length of pruned tree
    actual_length = tree_length(pruned_tree)
    print(f"Actual branch length of pruned tree: {actual_length}")
    
    # Compare with a random selection of k leaves
    import random
    random.seed(42)
    all_leaves = yule_tree.get_leaves()
    random_leaves = random.sample([leaf.name for leaf in all_leaves], k)
    random_tree = prune_tree_with_leaves(yule_tree, random_leaves)
    random_length = tree_length(random_tree)
    
    print("\nComparison:")
    print(f"Optimal pruned tree length: {actual_length}")
    print(f"Random pruned tree length: {random_length}")
    print(f"Improvement: {(random_length - actual_length) / random_length * 100:.2f}%")

# Minimum pairwise distance subtree
21 mars 2025 : on essaye de trouver le sous-arbre avec la somme de distances pairwise la plus faible

In [ ]:
import dendropy
from ete3 import Tree
import math
import itertools

def give_depth(tree):
    # First assign depth 0 to leaves.
    for node in tree.traverse(strategy="postorder"):
        if node.is_leaf():
            node.depth = 0
    # Then assign depth for internal nodes: depth = max(child.depth + child.dist)
    for node in tree.traverse(strategy="postorder"):
        if not node.is_leaf():
            node.depth = max(child.depth + child.dist for child in node.get_children())


def min_pairwise_sum(tree, k, take_root=True):
    """Outputs the set of k leaves such that the resulting tree has minimal sum
    of pairwise distances. The tree is assumed to be ultrametric.
    If take_root is True, we constrain the root to be in the selected set."""


    give_depth(tree)
    
    # dp will be a dictionary {node: {nb_spld: (min_cost, leaves_list)}}
    dp = {}
    
    def compute_dp(node):
        # Initialize dp table for node with keys 0..k.
        dp_node = {n_spld: (math.inf, []) for n_spld in range(k+1)}
        
        # Base case: if node is a leaf.
        if node.is_leaf():
            dp_node[0] = (0, [])
            dp_node[1] = (0, [node])
            dp[node] = dp_node
            return dp_node
        
        children = node.get_children()
        left, right = children[0], children[1]
        dp_left = compute_dp(left)
        dp_right = compute_dp(right)
        

        node_depth = left.dist + left.depth
        
        if node.is_root() and take_root:
            valid_j = lambda i, j: (i >= 2 and j >= 1 and (i - j) >= 1)
        else:
            valid_j = lambda i, j: True
        
        # Merge the two DP tables.
        for i in range(k+1):
            best_cost = math.inf
            best_leaves = []
            # Consider all splits j + (i-j) = i.
            for j in range(i+1):
                if not valid_j(i, j):
                    continue
                cost_left, leaves_left = dp_left.get(j, (math.inf, []))
                cost_right, leaves_right = dp_right.get(i - j, (math.inf, []))
                if cost_left == math.inf or cost_right == math.inf:
                    continue

                left_to_right_paths = j * (i - j) * (2 * node_depth)
                total_cost = cost_left + cost_right + left_to_right_paths
                if total_cost < best_cost:
                    best_cost = total_cost
                    best_leaves = leaves_left + leaves_right
            dp_node[i] = (best_cost, best_leaves)
        dp[node] = dp_node
        return dp_node
    

    dp_table = compute_dp(tree)
    min_cost, leaves_list = dp_table.get(k, (math.inf, []))
    return min_cost, leaves_list

def enumerate_subsets_dp_sanity(tree, k):
    """
    For the sanity check: enumerate over all subsets of k leaves (from the tree's inherent root)
    such that at least one leaf is taken from each of the two children of the root,
    and return the minimum sum of pairwise distances and the subset achieving that.
    """
    # The tree's inherent root.
    root = tree
    children = root.get_children()
    if len(children) != 2:
        raise ValueError("Sanity check expects a binary tree with exactly two children at the root.")
    left_leaves = children[0].get_leaves()
    right_leaves = children[1].get_leaves()
    
    best_cost = math.inf
    best_subset = None
    # For each possible split: at least one from left, at least one from right.
    for i in range(1, k):
        if i > len(left_leaves) or (k - i) > len(right_leaves):
            continue  # not enough leaves on one side.
        for left_subset in itertools.combinations(left_leaves, i):
            for right_subset in itertools.combinations(right_leaves, k - i):
                subset = list(left_subset) + list(right_subset)
                # Compute the sum of pairwise distances.
                cost = 0
                for u, v in itertools.combinations(subset, 2):
                    cost += tree.get_distance(u, v)
                if cost < best_cost:
                    best_cost = cost
                    best_subset = subset
    return best_cost, best_subset

n_trials = 100
total_leaves = 12
sampled_leaved = 6
for i in range(n_trials):
    tree = dendropy.model.birthdeath.birth_death_tree(birth_rate=1, death_rate=0, num_extant_tips=total_leaves)
    tree.write(path="tree.nwk", schema="newick", suppress_rooting=True)
    # Example Newick string of an ultrametric tree (binary tree).
    # In an ultrametric tree, all leaves are equidistant from the root.
    tree = Tree("/home/enzo/Documents/git/WP1/DeepGhosts/bin/tree.nwk", format=1)
    k = sampled_leaved  # number of leaves to select
    
    # Compute the DP solution using our function.
    dp_cost, dp_leaves = min_pairwise_sum(tree, k, take_root=True)
    # Sanity check: enumerate over all valid subsets.
    sanity_cost, sanity_subset = enumerate_subsets_dp_sanity(tree, k)
    print(f"DP solution: {dp_cost}, Sanity check: {sanity_cost}")
    
    # Compare the two.
    if abs(dp_cost - sanity_cost) > 1e-9:
        raise ValueError("Sanity check FAILED!")
    print("--------------------------------------------------")

In [ ]:
!pip install pyqt5

In [ ]:
from ete3 import TreeStyle

# Test du pipeline complet
21 mars 2025

Etape 1: générer les arbres d'espèces.

In [ ]:
from pathlib import Path
import subprocess
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test")

In [ ]:
sp_tree_index = 0
complete_tree = OUTPUT_FOLDER / f"species_tree_{sp_tree_index}" / "complete_species_tree.nwk"
extant_tree = OUTPUT_FOLDER / f"species_tree_{sp_tree_index}" / "extant_species_tree.nwk"

In [ ]:
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/Zombi_ale_friendly/Parameters/SpeciesTreeParameters.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

24 mars

In [ ]:
import sys
import os
import re

def modify_parameters(input_file_path: str,
                             output_file_path: str,
                             replacements: dict):
    try:
        # Read the file content
        with open(input_file_path, 'r') as file:
            content = file.read()

        # Replace all parameters ending with '_SNAKEMAKE'
        for key, value in replacements.items():
            pattern = re.compile(rf"{key}_SNAKEMAKE")
            content = pattern.sub(value, content)

        # Write the modified content to the output file
        with open(output_file_path, 'w') as file:
            file.write(content)

        print(f"Successfully modified the file and saved to: {output_file_path}")
    except FileNotFoundError:
        print(f"Error: The file {input_file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")



input_file_path = '/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParametersTemplate.tsv'
output_file_path = "/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv"

# Parse replacement arguments
replacements_genome = {"DUPLICATION_RATE": "0.1",
                "LOSS_RATE": "0.2",
                "TRANSFER_RATE": "0.3",
                "ASSORTATIVE_TRANSFER": "False",
                "ALPHA": "10",
                "N_GENES": "1000",
                "SEED": "4268"}


modify_parameters(input_file_path, output_file_path, replacements_genome)

In [ ]:
replacements_species = {"SPECIATION": "1",
                        "EXTINCTION": "0",
                        "MAX_LINEAGES": "10",
                        "SEED": "4268"}
input_file_path = '/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParametersTemplate.tsv'
output_file_path = "/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv"
modify_parameters(input_file_path, output_file_path, replacements_species)

# Transferts préférentiels vers les espèces proches
24 mars

On doit faire un code qui permet de biaiser la probabilité de transférer vers des espèces proches phylogénétiquement.

In [ ]:
from ete3 import Tree
tree_path = "/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_0/T/CompleteTree.nwk"
tree = Tree(tree_path, format=1)

In [ ]:
def compute_total_branch_length(tree):
    total_length = 0
    for node in tree.traverse():
        total_length += node.dist
    return total_length

In [ ]:
def compute_sampled_length(tree, sampled_leaves):
    """Compute the sum of branch lengths that the sampled tree will have."""
    sampled_tree = tree.copy()
    sampled_tree = sampled_tree.prune(sampled_leaves, preserve_branch_length=True)
    total_length = compute_total_branch_length(sampled_tree)
    return total_length

D'abord, on calcule les taux de transfert de toutes les espèces données, en les normalisant par la somme des longueurs de branches des arbres échantillonnés.

On normalise ensuite l'arbre d'espèces, en divisant sa taille par la somme des longueurs de branches de l'arbre d'espèces complet.

In [ ]:
tree.get_farthest_leaf()

In [ ]:
extant_tree_path = "/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_0/T/ExtantTree.nwk"
extant_tree = Tree(extant_tree_path, format=1)

In [ ]:
def normalize_tree_height(tree):
    """Normalize the tree such that its height is 1."""
    copy_tree = tree.copy()
    # We get the distance from the root to the farthest leaf, without counting the length of the root node.
    max_height = copy_tree.get_farthest_leaf()[1] 
    for node in copy_tree.traverse():
        node.dist /= max_height
    return copy_tree

In [ ]:
def compute_pairwise_distances(tree):
    # compute pairwise distances between the species in the tree
    pairwise_distances = {}
    for leaf1 in tree:
        for leaf2 in tree:
            if leaf1 != leaf2:
                pairwise_distances[(leaf1.name, leaf2.name)] = tree.get_distance(leaf1, leaf2)
    return pairwise_distances

distances = compute_pairwise_distances(tree)

In [ ]:
import math
def exponentiate_distances(distances, alpha):
    # exponentiate the pairwise distances
    exponentiated_distances = {}
    for pair, distance in distances.items():
        exponentiated_distances[pair] = math.exp(-alpha * distance)
    return exponentiated_distances


# On crée maintenant les fichiers.
1. Taux de transfert des branches:
On voulait normaliser le taux de transfert des branches. En effet, après avoir fait plusieurs types de sampling (60 feuilles parmi 120, 60 feuilles parmi 300, etc...), nous avons remarqué que plus le nombre total d'espèces était grand, plus le nombre total de transferts reçus par l'arbre samplé était grand. Cela est en fait dû au fait que lorsqu'on échantillonne un arbre d'espèces avec 60 feuilles d'un arbre plus grand, la hauteur totale de cet arbre d'espèces dépend du nombre total d'espèces au total.

Est-ce qu'on peut calculer l'espérance de la longueur de l'arbre samplé grâce aux formules de Stadler ? 

In [ ]:
import pandas as pd
# 1. 

# 1. On normalise l'arbre par sa hauteur.
normalized_tree = normalize_tree_height(tree)


Il faut utiliser Zombi pour faire les transferts préférentiels entre espèces, et calculer les probabilités de transfert pairwise avec les distances au moment du transfert.

C'est trop lent avec la version de Zombi de base, donc on doit trouver comment accélérer cette opération.

Je crois que j'ai trouvé comment faire: il faut utiliser une propriété de l'opération softmax. Soit $x = (x_1, ..., x_n)$. Alors
$$\mathrm{softmax}(x) = \left(\frac{\exp(x_1)}{\sum_{i=1}^{n}{\exp(x_i)}}, \frac{\exp(x_2)}{\sum_{i=1}^{n}{\exp(x_i)}}, ..., \frac{\exp(x_n)}{\sum_{i=1}^{n}{\exp(x_i)}}\right)$$

On remarque alors que $\mathrm{softmax}(x_1, ..., x_n) = \mathrm{softmax}(x_1+t, x_2+t, ..., x_3+t)$. Lorsque le temps augmente dans l'arbre complet de $dt$ sans passer un noeud, les distances pairwise entre le donneur et le receveur augmentent toutes de $2dt$. En vertu de l'équation $\mathrm{softmax}(x_1, ..., x_n) = \mathrm{softmax}(x_1+2dt, x_2+2dt, ..., x_3+2dt)$, on peut calculer les distances pairwise une fois pour toutes à chaque intervalle de la subdivision. Ainsi, on obtient un tenseur de taille $(n_intervalles, n_{feuilles}, n_{feuilles})$, soit de taille $O(n_{feuilles}^3)$, que l'on peut calculer une fois pour toutes et qui donnera les probabilités qu'un transfert donné soit reçu par une espèce contemporaine.

In [ ]:
# On teste les deux versions du code, pour voir si elles donnent le même résultat.
import sys
import os
import re

def modify_parameters(input_file_path: str,
                             output_file_path: str,
                             replacements: dict):
    try:
        # Read the file content
        with open(input_file_path, 'r') as file:
            content = file.read()

        # Replace all parameters ending with '_SNAKEMAKE'
        for key, value in replacements.items():
            pattern = re.compile(rf"{key}_SNAKEMAKE")
            content = pattern.sub(value, content)

        # Write the modified content to the output file
        with open(output_file_path, 'w+') as file:
            file.write(content)

        print(f"Successfully modified the file and saved to: {output_file_path}")
    except FileNotFoundError:
        print(f"Error: The file {input_file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")






replacements_species = {"SPECIATION": "1",
                        "EXTINCTION": "0",
                        "MAX_LINEAGES": "1000",
                        "SEED": "46"}
input_file_path = '/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParametersTemplate.tsv'
output_file_path = "/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv"
modify_parameters(input_file_path, output_file_path, replacements_species)
# Parse replacement arguments
input_file_path = '/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParametersTemplate.tsv'
output_file_path = "/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv"
replacements_genome = {"DUPLICATION_RATE": "0.0",
                "LOSS_RATE": "0.0",
                "TRANSFER_RATE": "25",
                "ASSORTATIVE_TRANSFER": "True",
                "ALPHA": "10",
                "N_GENES": "1000",
                "SEED": "48"}


modify_parameters(input_file_path, output_file_path, replacements_genome)

In [ ]:
1000*0.025

In [ ]:
120*0.025

In [ ]:
tree_output_folder == "/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_0"

In [ ]:
# Maintenant, on génère l'arbre d'espèces
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 0
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

Avec assortative_transfer=True. 67 minutes pour 1000 gènes, taux 25, 1000 espèces.

In [ ]:
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

Temps pour version d'Enzo:
10 minutes

In [ ]:
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

On vérifie maintenant que les deux versions donnent les mêmes valeurs.

In [270]:
replacements_species = {"SPECIATION": "1",
                        "EXTINCTION": "0",
                        "MAX_LINEAGES": "20",
                        "SEED": "46"}
input_file_path = '/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParametersTemplate.tsv'
output_file_path = "/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv"
modify_parameters(input_file_path, output_file_path, replacements_species)
# Parse replacement arguments
input_file_path = '/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParametersTemplate.tsv'
output_file_path = "/home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv"
replacements_genome = {"DUPLICATION_RATE": "0.0",
                "LOSS_RATE": "0.0",
                "TRANSFER_RATE": "25",
                "ASSORTATIVE_TRANSFER": "True",
                "ALPHA": "10",
                "N_GENES": "10",
                "SEED": "46"}


modify_parameters(input_file_path, output_file_path, replacements_genome)

Successfully modified the file and saved to: /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv
Successfully modified the file and saved to: /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv


In [272]:
# Maintenant, on génère l'arbre d'espèces
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 0
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

Computing Species Tree. Trial number 1
Not testing Enzo
Simulating genomes. Time 0.0
Simulating genomes. Time 0.02836572470159681
Simulating genomes. Time 0.05059514312230262
Simulating genomes. Time 0.1313158765295284
Simulating genomes. Time 0.13571179261634186
Simulating genomes. Time 0.15580175619922215
Simulating genomes. Time 0.15726784020760345
Simulating genomes. Time 0.15922088202349907
Simulating genomes. Time 0.1713958504028908
Simulating genomes. Time 0.19150653418741667
Simulating genomes. Time 0.20150149698624756
Simulating genomes. Time 0.20388619770984973
Simulating genomes. Time 0.21459878164594376
Simulating genomes. Time 0.25284933356496186
Simulating genomes. Time 0.25662550844464843
Simulating genomes. Time 0.30128535431891423
Simulating genomes. Time 0.30222525111347776
Simulating genomes. Time 0.30461743038160044
Simulating genomes. Time 0.3072887948035892
Simulating genomes. Time 0.3147230794790639
Simulating genomes. Time 0.31922998243171874
Simulating genomes.

CompletedProcess(args='python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv /home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_0', returncode=0)

In [256]:
import pandas as pd
from glob import glob
import os
species_tree_index = 0
# open every tsv file of the form *_branchevents.tsv in the folder, and concatenate them into a single dataframe with a new columns containing the * part of the filename.
folder_path = f"/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_{species_tree_index}/G/Gene_families/"
all_files = glob(os.path.join(folder_path, "*_events.tsv"))
df_from_each_file = (pd.read_csv(f, sep="\t").assign(filename=os.path.basename(f).split("_")[0]) for f in all_files)
events_df_0 = pd.concat(df_from_each_file, ignore_index=True)

In [348]:
# Maintenant, on génère l'arbre d'espèces
from IPython.display import clear_output
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 0
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
clear_output()
import pandas as pd
from glob import glob
import os
species_tree_index = 0
# open every tsv file of the form *_branchevents.tsv in the folder, and concatenate them into a single dataframe with a new columns containing the * part of the filename.
folder_path = f"/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_{species_tree_index}/G/Gene_families/"
all_files = glob(os.path.join(folder_path, "*_events.tsv"))
df_from_each_file = (pd.read_csv(f, sep="\t").assign(filename=os.path.basename(f).split("_")[0]) for f in all_files)
events_df_0 = pd.concat(df_from_each_file, ignore_index=True)
events_df_0[events_df_0["EVENT"]=="T"].sort_values(by="TIME")

,TIME,EVENT,NODES,filename
289,1.535556,T,100002;3;100002;4;100001;5,4
753,1.563927,T,100001;2;100001;4;100002;5,7
175,1.565400,T,100001;2;100001;4;100002;5,1
81,1.603259,T,100001;2;100001;4;100002;5,2
83,1.635550,T,100002;5;100002;6;100001;7,2
...,...,...,...,...
835,3.021590,T,100034;87;100034;104;100019;105,7
265,3.021679,T,100035;111;100035;112;100020;113,1
363,3.026598,T,100034;91;100034;96;100023;97,4
949,3.027679,T,100028;75;100028;112;100031;113,10


In [355]:
# Maintenant, on génère l'arbre d'espèces
from IPython.display import clear_output
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 1
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
#clear_output()
import pandas as pd
from glob import glob
import os
species_tree_index = 1
# open every tsv file of the form *_branchevents.tsv in the folder, and concatenate them into a single dataframe with a new columns containing the * part of the filename.
folder_path = f"/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_{species_tree_index}/G/Gene_families/"
all_files = glob(os.path.join(folder_path, "*_events.tsv"))
df_from_each_file = (pd.read_csv(f, sep="\t").assign(filename=os.path.basename(f).split("_")[0]) for f in all_files)
events_df_1 = pd.concat(df_from_each_file, ignore_index=True)
events_df_1[events_df_1["EVENT"]=="T"].sort_values(by="TIME")

Computing Species Tree. Trial number 1
Setting seed to 46
Simulating genomes. Time 0.0
Simulating genomes. Time 0.02836572470159681
Simulating genomes. Time 0.05059514312230262
Simulating genomes. Time 0.1313158765295284
Simulating genomes. Time 0.13571179261634186
Simulating genomes. Time 0.15580175619922215
Simulating genomes. Time 0.15726784020760345
Simulating genomes. Time 0.15922088202349907
Simulating genomes. Time 0.1713958504028908
Simulating genomes. Time 0.19150653418741667
Simulating genomes. Time 0.20150149698624756
Simulating genomes. Time 0.20388619770984973
Simulating genomes. Time 0.21459878164594376
Simulating genomes. Time 0.25284933356496186
Simulating genomes. Time 0.25662550844464843
Simulating genomes. Time 0.30128535431891423
Simulating genomes. Time 0.30222525111347776
Simulating genomes. Time 0.30461743038160044
Simulating genomes. Time 0.3072887948035892
Simulating genomes. Time 0.3147230794790639
Simulating genomes. Time 0.31922998243171874
Simulating genome

Traceback (most recent call last):
  File "/home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py", line 443, in <module>
    Z.G(parameters_file, experiment_folder, advanced_mode)
  File "/home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py", line 146, in G
    gss.run()
  File "/home/enzo/Documents/git/Zombi_ale_friendly/GenomeSimulator.py", line 503, in run
    self.evolve_genomes(d, t, l, i, c, o, current_time)
  File "/home/enzo/Documents/git/Zombi_ale_friendly/GenomeSimulator.py", line 899, in evolve_genomes
    self.make_transfer(t_e, donor, recipient, time)
    ^^^^^^^^^^^^^^^^^^
AttributeError: 'GenomeSimulator' object has no attribute 'make_transfer'


ValueError: No objects to concatenate

In [ ]:
events_df_1[events_df_1["EVENT"]=="T"].sort_values(by="TIME") == events_df_0[events_df_0["EVENT"]=="T"].sort_values(by="TIME")

,TIME,EVENT,NODES,filename
289,True,True,True,True
753,True,True,True,True
175,True,True,True,True
81,True,True,True,True
83,True,True,True,True
...,...,...,...,...
835,True,True,True,True
265,True,True,True,True
363,True,True,True,True
949,True,True,True,True


In [316]:
import numpy as np
l1 = [('100002', np.str_('100001')), ('100001', np.str_('100002')), ('100001', np.str_('100002')), ('100001', np.str_('100002')), ('100002', np.str_('100001')), ('100001', np.str_('100002')), ('100001', np.str_('100002')), ('100004', np.str_('100003')), ('100004', np.str_('100003')), ('100001', np.str_('100004')), ('100001', np.str_('100004')), ('100001', np.str_('100004')), ('100001', np.str_('100003')), ('100003', np.str_('100004')), ('100001', np.str_('100003')), ('100003', np.str_('100004')), ('100004', np.str_('100003')), ('100003', np.str_('100007')), ('100003', np.str_('100007')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100006', np.str_('100005')), ('100008', np.str_('100007')), ('100007', np.str_('100005')), ('100007', np.str_('100003')), ('100006', np.str_('100005')), ('100003', np.str_('100005')), ('100008', np.str_('100007')), ('100006', np.str_('100005')), ('100007', np.str_('100006')), ('100008', np.str_('100007')), ('100007', np.str_('100008')), ('100008', np.str_('100007')), ('100007', np.str_('100008')), ('100005', np.str_('100008')), ('100005', np.str_('100006')), ('100005', np.str_('100003')), ('100008', np.str_('100007')), ('100005', np.str_('100006')), ('100005', np.str_('100006')), ('100003', np.str_('100007')), ('100007', np.str_('100008')), ('100006', np.str_('100005')), ('100005', np.str_('100003')), ('100005', np.str_('100008')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100008', np.str_('100007')), ('100008', np.str_('100007')), ('100006', np.str_('100005')), ('100006', np.str_('100005')), ('100007', np.str_('100008')), ('100005', np.str_('100006')), ('100006', np.str_('100005')), ('100005', np.str_('100006')), ('100006', np.str_('100005')), ('100003', np.str_('100006')), ('100008', np.str_('100007')), ('100005', np.str_('100008')), ('100006', np.str_('100008')), ('100006', np.str_('100005')), ('100005', np.str_('100006')), ('100007', np.str_('100008')), ('100006', np.str_('100005')), ('100006', np.str_('100005')), ('100007', np.str_('100008')), ('100005', np.str_('100007')), ('100005', np.str_('100003')), ('100008', np.str_('100003')), ('100007', np.str_('100008')), ('100008', np.str_('100007')), ('100005', np.str_('100006')), ('100007', np.str_('100003')), ('100003', np.str_('100006')), ('100006', np.str_('100005')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100007', np.str_('100003')), ('100005', np.str_('100007')), ('100003', np.str_('100008')), ('100003', np.str_('100007')), ('100008', np.str_('100007')), ('100003', np.str_('100008')), ('100003', np.str_('100008')), ('100003', np.str_('100007')), ('100008', np.str_('100007')), ('100007', np.str_('100003')), ('100003', np.str_('100005')), ('100009', np.str_('100007')), ('100003', np.str_('100010')), ('100007', np.str_('100009')), ('100007', np.str_('100009')), ('100006', np.str_('100003')), ('100003', np.str_('100010')), ('100007', np.str_('100005')), ('100006', np.str_('100005')), ('100005', np.str_('100007')), ('100010', np.str_('100005')), ('100007', np.str_('100009')), ('100010', np.str_('100009')), ('100006', np.str_('100007')), ('100009', np.str_('100003')), ('100003', np.str_('100007')), ('100005', np.str_('100010')), ('100006', np.str_('100011')), ('100010', np.str_('100009')), ('100003', np.str_('100009')), ('100010', np.str_('100006')), ('100007', np.str_('100010')), ('100011', np.str_('100003')), ('100010', np.str_('100009')), ('100003', np.str_('100006')), ('100007', np.str_('100012')), ('100012', np.str_('100011')), ('100010', np.str_('100009')), ('100003', np.str_('100006')), ('100003', np.str_('100009')), ('100012', np.str_('100011')), ('100003', np.str_('100010')), ('100009', np.str_('100007')), ('100013', np.str_('100009')), ('100014', np.str_('100007')), ('100012', np.str_('100013')), ('100007', np.str_('100012')), ('100003', np.str_('100006')), ('100009', np.str_('100014')), ('100006', np.str_('100012')), ('100011', np.str_('100006')), ('100006', np.str_('100011')), ('100013', np.str_('100006')), ('100009', np.str_('100011')), ('100007', np.str_('100003')), ('100011', np.str_('100012')), ('100003', np.str_('100013')), ('100011', np.str_('100012')), ('100011', np.str_('100012')), ('100011', np.str_('100012')), ('100007', np.str_('100009')), ('100013', np.str_('100009')), ('100003', np.str_('100006')), ('100007', np.str_('100011')), ('100014', np.str_('100003')), ('100014', np.str_('100007')), ('100012', np.str_('100011')), ('100012', np.str_('100011')), ('100009', np.str_('100007')), ('100012', np.str_('100011')), ('100013', np.str_('100009')), ('100012', np.str_('100011')), ('100013', np.str_('100009')), ('100006', np.str_('100012')), ('100013', np.str_('100009')), ('100007', np.str_('100006')), ('100009', np.str_('100014')), ('100013', np.str_('100007')), ('100009', np.str_('100007')), ('100014', np.str_('100009')), ('100012', np.str_('100011')), ('100007', np.str_('100006')), ('100014', np.str_('100013')), ('100014', np.str_('100013')), ('100007', np.str_('100014')), ('100006', np.str_('100009')), ('100011', np.str_('100007')), ('100013', np.str_('100014')), ('100007', np.str_('100013')), ('100011', np.str_('100014')), ('100012', np.str_('100013')), ('100012', np.str_('100006')), ('100007', np.str_('100003')), ('100003', np.str_('100012')), ('100011', np.str_('100012')), ('100007', np.str_('100013')), ('100015', np.str_('100014')), ('100018', np.str_('100011')), ('100015', np.str_('100011')), ('100014', np.str_('100013')), ('100016', np.str_('100012')), ('100013', np.str_('100009')), ('100018', np.str_('100014')), ('100016', np.str_('100012')), ('100014', np.str_('100015')), ('100012', np.str_('100011')), ('100007', np.str_('100013')), ('100013', np.str_('100009')), ('100011', np.str_('100007')), ('100012', np.str_('100020')), ('100007', np.str_('100022')), ('100018', np.str_('100022')), ('100013', np.str_('100019')), ('100013', np.str_('100020')), ('100013', np.str_('100019')), ('100014', np.str_('100017')), ('100017', np.str_('100011')), ('100014', np.str_('100007')), ('100012', np.str_('100014')), ('100017', np.str_('100018')), ('100016', np.str_('100024')), ('100024', np.str_('100019')), ('100026', np.str_('100014')), ('100012', np.str_('100011')), ('100019', np.str_('100020')), ('100017', np.str_('100014')), ('100013', np.str_('100027')), ('100022', np.str_('100025')), ('100011', np.str_('100017')), ('100025', np.str_('100022')), ('100028', np.str_('100016')), ('100020', np.str_('100012')), ('100022', np.str_('100014')), ('100024', np.str_('100020')), ('100024', np.str_('100020')), ('100028', np.str_('100012')), ('100018', np.str_('100022')), ('100013', np.str_('100027')), ('100025', np.str_('100016')), ('100023', np.str_('100018')), ('100025', np.str_('100031')), ('100027', np.str_('100019')), ('100019', np.str_('100020')), ('100019', np.str_('100014')), ('100014', np.str_('100013')), ('100029', np.str_('100012')), ('100031', np.str_('100012')), ('100032', np.str_('100020')), ('100013', np.str_('100025')), ('100028', np.str_('100027')), ('100028', np.str_('100029')), ('100031', np.str_('100029')), ('100013', np.str_('100032')), ('100016', np.str_('100020')), ('100032', np.str_('100019')), ('100014', np.str_('100020')), ('100031', np.str_('100027')), ('100025', np.str_('100032')), ('100023', np.str_('100033')), ('100012', np.str_('100018')), ('100025', np.str_('100020')), ('100032', np.str_('100028')), ('100032', np.str_('100031')), ('100019', np.str_('100014')), ('100025', np.str_('100018')), ('100031', np.str_('100033')), ('100034', np.str_('100027')), ('100016', np.str_('100031')), ('100029', np.str_('100031')), ('100028', np.str_('100018')), ('100028', np.str_('100018')), ('100030', np.str_('100023')), ('100034', np.str_('100023')), ('100034', np.str_('100036')), ('100035', np.str_('100034')), ('100028', np.str_('100019')), ('100029', np.str_('100034')), ('100014', np.str_('100025')), ('100019', np.str_('100029')), ('100031', np.str_('100025')), ('100037', np.str_('100022')), ('100018', np.str_('100016')), ('100032', np.str_('100013')), ('100016', np.str_('100036')), ('100022', np.str_('100020')), ('100025', np.str_('100019')), ('100016', np.str_('100023')), ('100030', np.str_('100013')), ('100037', np.str_('100014')), ('100020', np.str_('100013')), ('100035', np.str_('100019')), ('100029', np.str_('100022')), ('100031', np.str_('100013')), ('100012', np.str_('100032')), ('100029', np.str_('100018')), ('100018', np.str_('100031')), ('100037', np.str_('100038')), ('100035', np.str_('100023')), ('100012', np.str_('100035')), ('100036', np.str_('100035')), ('100028', np.str_('100018')), ('100034', np.str_('100019')), ('100035', np.str_('100020')), ('100034', np.str_('100023')), ('100028', np.str_('100031')), ('100014', np.str_('100018'))]


In [317]:
l2 = [('100002', np.str_('100001')), ('100001', np.str_('100002')), ('100001', np.str_('100002')), ('100001', np.str_('100002')), ('100002', np.str_('100001')), ('100001', np.str_('100002')), ('100001', np.str_('100002')), ('100004', np.str_('100003')), ('100004', np.str_('100003')), ('100001', np.str_('100004')), ('100001', np.str_('100004')), ('100001', np.str_('100004')), ('100001', np.str_('100003')), ('100003', np.str_('100004')), ('100001', np.str_('100003')), ('100003', np.str_('100004')), ('100004', np.str_('100003')), ('100003', np.str_('100007')), ('100003', np.str_('100007')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100006', np.str_('100005')), ('100008', np.str_('100007')), ('100007', np.str_('100005')), ('100007', np.str_('100003')), ('100006', np.str_('100005')), ('100003', np.str_('100005')), ('100008', np.str_('100007')), ('100006', np.str_('100005')), ('100007', np.str_('100006')), ('100008', np.str_('100007')), ('100007', np.str_('100008')), ('100008', np.str_('100007')), ('100007', np.str_('100008')), ('100005', np.str_('100008')), ('100005', np.str_('100006')), ('100005', np.str_('100003')), ('100008', np.str_('100007')), ('100005', np.str_('100006')), ('100005', np.str_('100006')), ('100003', np.str_('100007')), ('100007', np.str_('100008')), ('100006', np.str_('100005')), ('100005', np.str_('100003')), ('100005', np.str_('100008')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100008', np.str_('100007')), ('100008', np.str_('100007')), ('100006', np.str_('100005')), ('100006', np.str_('100005')), ('100007', np.str_('100008')), ('100005', np.str_('100006')), ('100006', np.str_('100005')), ('100005', np.str_('100006')), ('100006', np.str_('100005')), ('100003', np.str_('100006')), ('100008', np.str_('100007')), ('100005', np.str_('100008')), ('100006', np.str_('100008')), ('100006', np.str_('100005')), ('100005', np.str_('100006')), ('100007', np.str_('100008')), ('100006', np.str_('100005')), ('100006', np.str_('100005')), ('100007', np.str_('100008')), ('100005', np.str_('100007')), ('100005', np.str_('100003')), ('100008', np.str_('100003')), ('100007', np.str_('100008')), ('100008', np.str_('100007')), ('100005', np.str_('100006')), ('100007', np.str_('100003')), ('100003', np.str_('100006')), ('100006', np.str_('100005')), ('100006', np.str_('100005')), ('100003', np.str_('100007')), ('100007', np.str_('100003')), ('100005', np.str_('100007')), ('100003', np.str_('100008')), ('100003', np.str_('100007')), ('100008', np.str_('100007')), ('100003', np.str_('100008')), ('100003', np.str_('100008')), ('100003', np.str_('100007')), ('100008', np.str_('100007')), ('100007', np.str_('100003')), ('100003', np.str_('100005')), ('100009', np.str_('100007')), ('100003', np.str_('100010')), ('100007', np.str_('100009')), ('100007', np.str_('100009')), ('100006', np.str_('100003')), ('100003', np.str_('100010')), ('100007', np.str_('100005')), ('100006', np.str_('100005')), ('100005', np.str_('100007')), ('100010', np.str_('100005')), ('100007', np.str_('100009')), ('100010', np.str_('100009')), ('100006', np.str_('100007')), ('100009', np.str_('100003')), ('100003', np.str_('100007')), ('100005', np.str_('100010')), ('100006', np.str_('100011')), ('100010', np.str_('100009')), ('100003', np.str_('100009')), ('100010', np.str_('100006')), ('100007', np.str_('100010')), ('100011', np.str_('100003')), ('100010', np.str_('100009')), ('100003', np.str_('100006')), ('100007', np.str_('100012')), ('100012', np.str_('100011')), ('100010', np.str_('100009')), ('100003', np.str_('100006')), ('100003', np.str_('100009')), ('100012', np.str_('100011')), ('100003', np.str_('100010')), ('100009', np.str_('100007')), ('100013', np.str_('100009')), ('100014', np.str_('100007')), ('100012', np.str_('100013')), ('100007', np.str_('100012')), ('100003', np.str_('100006')), ('100009', np.str_('100014')), ('100006', np.str_('100012')), ('100011', np.str_('100006')), ('100006', np.str_('100011')), ('100013', np.str_('100006')), ('100009', np.str_('100011')), ('100007', np.str_('100003')), ('100011', np.str_('100012')), ('100003', np.str_('100013')), ('100011', np.str_('100012')), ('100011', np.str_('100012')), ('100011', np.str_('100012')), ('100007', np.str_('100009')), ('100013', np.str_('100009')), ('100003', np.str_('100006')), ('100007', np.str_('100011')), ('100014', np.str_('100003')), ('100014', np.str_('100007')), ('100012', np.str_('100011')), ('100012', np.str_('100011')), ('100009', np.str_('100007')), ('100012', np.str_('100011')), ('100013', np.str_('100009')), ('100012', np.str_('100011')), ('100013', np.str_('100009')), ('100006', np.str_('100012')), ('100013', np.str_('100009')), ('100007', np.str_('100006')), ('100009', np.str_('100014')), ('100013', np.str_('100007')), ('100009', np.str_('100007')), ('100014', np.str_('100009')), ('100012', np.str_('100011')), ('100007', np.str_('100006')), ('100014', np.str_('100013')), ('100014', np.str_('100013')), ('100007', np.str_('100014')), ('100006', np.str_('100009')), ('100011', np.str_('100007')), ('100013', np.str_('100014')), ('100007', np.str_('100013')), ('100011', np.str_('100014')), ('100012', np.str_('100013')), ('100012', np.str_('100006')), ('100007', np.str_('100003')), ('100003', np.str_('100012')), ('100011', np.str_('100012')), ('100007', np.str_('100013')), ('100015', np.str_('100014')), ('100018', np.str_('100011')), ('100015', np.str_('100011')), ('100014', np.str_('100013')), ('100016', np.str_('100012')), ('100013', np.str_('100009')), ('100018', np.str_('100014')), ('100016', np.str_('100012')), ('100014', np.str_('100015')), ('100012', np.str_('100011')), ('100007', np.str_('100013')), ('100013', np.str_('100009')), ('100011', np.str_('100007')), ('100012', np.str_('100020')), ('100007', np.str_('100022')), ('100018', np.str_('100022')), ('100013', np.str_('100019')), ('100013', np.str_('100020')), ('100013', np.str_('100019')), ('100014', np.str_('100017')), ('100017', np.str_('100011')), ('100014', np.str_('100007')), ('100012', np.str_('100014')), ('100017', np.str_('100018')), ('100016', np.str_('100024')), ('100024', np.str_('100019')), ('100026', np.str_('100014')), ('100012', np.str_('100011')), ('100019', np.str_('100020')), ('100017', np.str_('100014')), ('100013', np.str_('100027')), ('100022', np.str_('100025')), ('100011', np.str_('100017')), ('100025', np.str_('100022')), ('100028', np.str_('100016')), ('100020', np.str_('100012')), ('100022', np.str_('100014')), ('100024', np.str_('100020')), ('100024', np.str_('100020')), ('100028', np.str_('100012')), ('100018', np.str_('100022')), ('100013', np.str_('100027')), ('100025', np.str_('100016')), ('100023', np.str_('100018')), ('100025', np.str_('100031')), ('100027', np.str_('100019')), ('100019', np.str_('100020')), ('100019', np.str_('100014')), ('100014', np.str_('100013')), ('100029', np.str_('100012')), ('100031', np.str_('100012')), ('100032', np.str_('100020')), ('100013', np.str_('100025')), ('100028', np.str_('100027')), ('100028', np.str_('100029')), ('100031', np.str_('100029')), ('100013', np.str_('100032')), ('100016', np.str_('100020')), ('100032', np.str_('100019')), ('100014', np.str_('100020')), ('100031', np.str_('100027')), ('100025', np.str_('100032')), ('100023', np.str_('100033')), ('100012', np.str_('100018')), ('100025', np.str_('100020')), ('100032', np.str_('100028')), ('100032', np.str_('100031')), ('100019', np.str_('100014')), ('100025', np.str_('100018')), ('100031', np.str_('100033')), ('100034', np.str_('100027')), ('100016', np.str_('100031')), ('100029', np.str_('100031')), ('100028', np.str_('100018')), ('100028', np.str_('100018')), ('100030', np.str_('100023')), ('100034', np.str_('100023')), ('100034', np.str_('100036')), ('100035', np.str_('100034')), ('100028', np.str_('100019')), ('100029', np.str_('100034')), ('100014', np.str_('100025')), ('100019', np.str_('100029')), ('100031', np.str_('100025')), ('100037', np.str_('100022')), ('100018', np.str_('100016')), ('100032', np.str_('100013')), ('100016', np.str_('100036')), ('100022', np.str_('100020')), ('100025', np.str_('100019')), ('100016', np.str_('100023')), ('100030', np.str_('100013')), ('100037', np.str_('100014')), ('100020', np.str_('100013')), ('100035', np.str_('100019')), ('100029', np.str_('100022')), ('100031', np.str_('100013')), ('100012', np.str_('100032')), ('100029', np.str_('100018')), ('100018', np.str_('100031')), ('100037', np.str_('100038')), ('100035', np.str_('100023')), ('100012', np.str_('100035')), ('100036', np.str_('100035')), ('100028', np.str_('100018')), ('100034', np.str_('100019')), ('100035', np.str_('100020')), ('100034', np.str_('100023')), ('100028', np.str_('100031')), ('100014', np.str_('100018'))]


In [318]:
l1 == l2

True

In [297]:
[x[0] for x in l1] == [x[0] for x in l2]

False

In [368]:
# Maintenant, on génère l'arbre d'espèces
from IPython.display import clear_output
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 0
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
clear_output()
import pandas as pd
from glob import glob
import os
species_tree_index = 0
# open every tsv file of the form *_branchevents.tsv in the folder, and concatenate them into a single dataframe with a new columns containing the * part of the filename.
folder_path = f"/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_{species_tree_index}/G/Gene_families/"
all_files = glob(os.path.join(folder_path, "*_events.tsv"))
df_from_each_file = (pd.read_csv(f, sep="\t").assign(filename=os.path.basename(f).split("_")[0]) for f in all_files)
events_df_0 = pd.concat(df_from_each_file, ignore_index=True)
events_df_0[events_df_0["EVENT"]=="T"].sort_values(by="TIME")

,TIME,EVENT,NODES,filename
289,1.535556,T,100002;3;100002;4;100001;5,4
753,1.563927,T,100001;2;100001;4;100002;5,7
175,1.565400,T,100001;2;100001;4;100002;5,1
81,1.603259,T,100001;2;100001;4;100002;5,2
83,1.635550,T,100002;5;100002;6;100001;7,2
...,...,...,...,...
835,3.021590,T,100034;87;100034;104;100019;105,7
265,3.021679,T,100035;111;100035;112;100020;113,1
363,3.026598,T,100034;91;100034;96;100023;97,4
949,3.027679,T,100028;75;100028;112;100031;113,10


In [369]:
# Maintenant, on génère l'arbre d'espèces
from IPython.display import clear_output
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 1
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
command = f"python3 /home/enzo/Documents/git/Zombi_ale_friendly/Zombi.py G /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)
clear_output()
import pandas as pd
from glob import glob
import os
species_tree_index = 1
# open every tsv file of the form *_branchevents.tsv in the folder, and concatenate them into a single dataframe with a new columns containing the * part of the filename.
folder_path = f"/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_{species_tree_index}/G/Gene_families/"
all_files = glob(os.path.join(folder_path, "*_events.tsv"))
df_from_each_file = (pd.read_csv(f, sep="\t").assign(filename=os.path.basename(f).split("_")[0]) for f in all_files)
events_df_1 = pd.concat(df_from_each_file, ignore_index=True)
events_df_1[events_df_1["EVENT"]=="T"].sort_values(by="TIME")

,TIME,EVENT,NODES,filename
289,1.535556,T,100002;3;100002;4;100001;5,4
753,1.563927,T,100001;2;100001;4;100002;5,7
175,1.565400,T,100001;2;100001;4;100002;5,1
81,1.603259,T,100001;2;100001;4;100002;5,2
83,1.635550,T,100002;5;100002;6;100001;7,2
...,...,...,...,...
835,3.021590,T,100034;87;100034;104;100019;105,7
265,3.021679,T,100035;111;100035;112;100020;113,1
363,3.026598,T,100034;91;100034;96;100023;97,4
949,3.027679,T,100028;75;100028;112;100031;113,10


In [359]:
events_df_0[events_df_0["EVENT"]=="T"].sort_values(by="TIME")

,TIME,EVENT,NODES,filename
289,1.535556,T,100002;3;100002;4;100001;5,4
753,1.563927,T,100001;2;100001;4;100002;5,7
175,1.565400,T,100001;2;100001;4;100002;5,1
81,1.603259,T,100001;2;100001;4;100002;5,2
83,1.635550,T,100002;5;100002;6;100001;7,2
...,...,...,...,...
835,3.021590,T,100034;87;100034;104;100019;105,7
265,3.021679,T,100035;111;100035;112;100020;113,1
363,3.026598,T,100034;91;100034;96;100023;97,4
949,3.027679,T,100028;75;100028;112;100031;113,10


In [371]:
events_df_0_T = events_df_0[events_df_0["EVENT"]=="T"].sort_values(by="TIME")
events_df_1_T = events_df_1[events_df_1["EVENT"]=="T"].sort_values(by="TIME")
events_df_0_T == events_df_1_T

,TIME,EVENT,NODES,filename
289,True,True,True,True
753,True,True,True,True
175,True,True,True,True
81,True,True,True,True
83,True,True,True,True
...,...,...,...,...
835,True,True,True,True
265,True,True,True,True
363,True,True,True,True
949,True,True,True,True


In [372]:
events_df_0_T.equals(events_df_1_T)

True

In [360]:
(events_df_0["TIME"] == events_df_1["TIME"]).all()

np.True_

In [373]:
events_df_0 == events_df_1

,TIME,EVENT,NODES,filename
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
4,True,True,True,True
...,...,...,...,...
965,True,True,False,True
966,True,True,False,True
967,True,True,False,True
968,True,True,False,True


In [377]:
events_df_0

,TIME,EVENT,NODES,filename
0,0.000000,O,100000,9
1,1.531701,S,100000;1;100001;2;100002;3,9
2,1.674905,S,100002;3;100003;4;100004;5,9
3,1.800082,S,100001;2;100005;6;100006;7,9
4,1.811035,S,100004;5;100007;8;100008;9,9
...,...,...,...,...
965,3.027820,F,100037;106,10
966,3.027820,F,100022;57,10
967,3.027820,F,100025;84,10
968,3.027820,F,100014;101,10


In [375]:
events_df_1

,TIME,EVENT,NODES,filename
0,0.000000,O,100000,9
1,1.531701,S,100000;1;100001;2;100002;3,9
2,1.674905,S,100002;3;100003;4;100004;5,9
3,1.800082,S,100001;2;100005;6;100006;7,9
4,1.811035,S,100004;5;100007;8;100008;9,9
...,...,...,...,...
965,3.027820,F,100016;43,10
966,3.027820,F,100032;97,10
967,3.027820,F,100028;112,10
968,3.027820,F,100013;109,10


In [378]:
import pandas as pd

# Assuming events_df_0 and events_df_1 are already defined
# Sorting the DataFrames by the specified columns
sorted_df_0 = events_df_0.sort_values(by=["TIME", "EVENT", "NODES", "filename"]).reset_index(drop=True)
sorted_df_1 = events_df_1.sort_values(by=["TIME", "EVENT", "NODES", "filename"]).reset_index(drop=True)

# Checking equality
dataframes_are_equal = sorted_df_0.equals(sorted_df_1)

dataframes_are_equal


True

In [379]:
sorted_df_1

,TIME,EVENT,NODES,filename
0,0.00000,O,100000,1
1,0.00000,O,100000,10
2,0.00000,O,100000,2
3,0.00000,O,100000,3
4,0.00000,O,100000,4
...,...,...,...,...
965,3.02782,F,100038;75,8
966,3.02782,F,100038;89,2
967,3.02782,F,100038;93,4
968,3.02782,F,100038;93,6


In [ ]:
events_df

In [ ]:
events_df["filename"].unique()

In [ ]:
events_df[events_df["EVENT"] == "T"]

In [ ]:
from ete3 import Tree
tree = Tree("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test/species_tree_0/T/CompleteTree.nwk", format=1)
sum([node.dist for node in tree.traverse()])

Test avec le Zombi de base

In [ ]:
!pip install pyvolve

In [ ]:
# Maintenant, on génère l'arbre d'espèces
import subprocess
from pathlib import Path
OUTPUT_FOLDER = Path("/home/enzo/Documents/git/WP1/DeepGhosts/Output_test") 
sp_tree_index = 1
tree_output_folder = str(OUTPUT_FOLDER / f"species_tree_{sp_tree_index}")
command = f"python3 /home/enzo/Documents/git/deeprec/Zombi/Zombi.py T /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/SpeciesTreeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)

In [ ]:
command = f"python3 /home/enzo/Documents/git/deeprec/Zombi/Zombi.py Gm /home/enzo/Documents/git/WP1/DeepGhosts/bin/zombi_configs/GenomeParameters_2.tsv {tree_output_folder}"
subprocess.run(command, shell=True)